In [ ]:
import os
import gdown
import zipfile

# Define COCO dataset URLs
coco_urls = {
    "train_images": "http://images.cocodataset.org/zips/train2017.zip",
    "val_images": "http://images.cocodataset.org/zips/val2017.zip",
    "annotations": "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
}

# Create a directory to store the dataset
dataset_path = "coco_dataset"
os.makedirs(dataset_path, exist_ok=True)

# Function to download and extract COCO data
def download_and_extract(url, save_path):
    zip_path = os.path.join(dataset_path, save_path)

    if not os.path.exists(zip_path):
        print(f"Downloading {save_path}...")
        gdown.download(url, zip_path, quiet=False)

        print(f"Extracting {save_path}...")
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(dataset_path)

        print(f"✅ {save_path} Downloaded & Extracted!")
    else:
        print(f"⚡ {save_path} already exists, skipping download.")

# Download and extract train, val, and annotations
for name, url in coco_urls.items():
    download_and_extract(url, f"{name}.zip")

print("\n🎉 COCO Dataset is ready!")


Downloading...
From: http://images.cocodataset.org/zips/train2017.zip
To: /content/coco_dataset/train_images.zip


100%|██████████| 19.3G/19.3G [07:12<00:00, 44.7MB/s]


Extracting train_images.zip...
✅ train_images.zip Downloaded & Extracted!


Downloading...
From: http://images.cocodataset.org/zips/val2017.zip
To: /content/coco_dataset/val_images.zip
100%|██████████| 816M/816M [00:15<00:00, 51.4MB/s]


Extracting val_images.zip...


Downloading...
From: http://images.cocodataset.org/annotations/annotations_trainval2017.zip
To: /content/coco_dataset/annotations.zip


✅ val_images.zip Downloaded & Extracted!


100%|██████████| 253M/253M [00:04<00:00, 54.7MB/s]


Extracting annotations.zip...
✅ annotations.zip Downloaded & Extracted!

🎉 COCO Dataset is ready!


In [ ]:
import os
import json
import shutil
from pycocotools.coco import COCO

# Define dataset paths
dataset_path = "/content/coco_dataset"
train_images_path = os.path.join(dataset_path, "train2017")
annotations_path = os.path.join(dataset_path, "annotations", "instances_train2017.json")

# Load COCO dataset annotations
coco = COCO(annotations_path)

# Define the 11 required object classes (names must match COCO dataset)
selected_classes = ["person", "bird", "cat", "dog", "cow", "horse",
                     "sheep", "elephant", "bear", "zebra", "giraffe"]

# Get correct category IDs for the selected classes
category_ids = coco.getCatIds(catNms=selected_classes)
print("Category IDs:", category_ids)

# Get image IDs that contain at least one of these categories
image_ids = set()
for cat_id in category_ids:
    img_ids_for_cat = coco.getImgIds(catIds=[cat_id])  # Get images for this category
    image_ids.update(img_ids_for_cat)  # Add them to the set (avoid duplicates)

image_ids = list(image_ids)  # Convert set to list
print(f"Total Images Found: {len(image_ids)}")

# Create a new directory to store filtered images
filtered_dataset_path = os.path.join(dataset_path, "filtered_train2017")
os.makedirs(filtered_dataset_path, exist_ok=True)

# Move filtered images to the new directory
for img_id in image_ids:
    img_info = coco.loadImgs(img_id)[0]
    img_filename = img_info["file_name"]

    src_path = os.path.join(train_images_path, img_filename)
    dst_path = os.path.join(filtered_dataset_path, img_filename)

    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)

print(f"✅ Filtered dataset is ready with {len(image_ids)} images!")


loading annotations into memory...
Done (t=17.42s)
creating index...
index created!
Category IDs: [1, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Total Images Found: 80585
✅ Filtered dataset is ready with 80585 images!


In [ ]:
import os
import json
from pycocotools.coco import COCO

# Paths
dataset_path = "/content/coco_dataset"
annotations_path = os.path.join(dataset_path, "annotations", "instances_train2017.json")
filtered_images_path = os.path.join(dataset_path, "filtered_train2017")
yolo_labels_path = os.path.join(dataset_path, "filtered_train_labels")

# Create directory for YOLO labels
os.makedirs(yolo_labels_path, exist_ok=True)

# Load COCO dataset
coco = COCO(annotations_path)

# Define COCO classes and corresponding YOLO class IDs
selected_classes = ["person", "bird", "cat", "dog", "cow", "horse",
                     "sheep", "elephant", "bear", "zebra", "giraffe"]

# Get COCO category IDs for selected classes
category_ids = coco.getCatIds(catNms=selected_classes)

# Map category IDs to YOLO class indices (0-10)
category_to_yolo = {cat_id: idx for idx, cat_id in enumerate(category_ids)}

# Get image IDs for selected classes
image_ids = coco.getImgIds(catIds=category_ids)

# Convert annotations for each image
for img_id in image_ids:
    img_info = coco.loadImgs(img_id)[0]
    img_filename = img_info["file_name"]
    img_width, img_height = img_info["width"], img_info["height"]

    # Get annotation data
    ann_ids = coco.getAnnIds(imgIds=img_id, catIds=category_ids, iscrowd=False)
    annotations = coco.loadAnns(ann_ids)

    # Prepare YOLO annotation file
    yolo_filename = os.path.splitext(img_filename)[0] + ".txt"
    yolo_filepath = os.path.join(yolo_labels_path, yolo_filename)

    with open(yolo_filepath, "w") as f:
        for ann in annotations:
            cat_id = ann["category_id"]
            if cat_id not in category_to_yolo:
                continue  # Skip if category is not selected

            # Get bounding box [x, y, width, height] and convert to YOLO format
            x, y, w, h = ann["bbox"]
            x_center = (x + w / 2) / img_width
            y_center = (y + h / 2) / img_height
            w /= img_width
            h /= img_height

            # Write to file (class_id, x_center, y_center, width, height)
            yolo_class_id = category_to_yolo[cat_id]
            f.write(f"{yolo_class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

print(f"✅ YOLO annotations saved in {yolo_labels_path}!")


In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet

# Fix CUDA-related issues for Colab
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/ARCH= -gencode arch=compute_75,code=sm_75/ARCH= -gencode arch=compute_70,code=sm_70/' Makefile

# Compile Darknet
!make


fatal: destination path 'darknet' already exists and is not an empty directory.
[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, flo

In [ ]:
!grep "filters=" /content/darknet/cfg/yolov3_custom.cfg
!grep "classes=" /content/darknet/cfg/yolov3_custom.cfg


In [ ]:
!./darknet


usage: ./darknet <function>


In [ ]:
# Create obj.names file using Python
classes = ["person", "bird", "cat", "dog", "cow", "horse",
           "sheep", "elephant", "bear", "zebra", "giraffe"]

with open("obj.names", "w") as f:
    for cls in classes:
        f.write(cls + "\n")

print("✅ obj.names file created successfully!")


✅ obj.names file created successfully!


In [ ]:
# Create obj.data file using Python
obj_data_content = """classes= 11
train= train.txt
valid= val.txt
names= obj.names
backup= backup/
"""

with open("obj.data", "w") as f:
    f.write(obj_data_content)

print("✅ obj.data file created successfully!")


✅ obj.data file created successfully!


In [ ]:
# Path to YOLOv3 config file
cfg_path = "/content/darknet/cfg/yolov3.cfg"

# Read the file and find [yolo] layers and filters before them
with open(cfg_path, "r") as f:
    lines = f.readlines()

yolo_indices = []
filters_indices = []

# Find all [yolo] layers and the filters before them
for i, line in enumerate(lines):
    if "[yolo]" in line:
        yolo_indices.append(i)
        # Find the previous [convolutional] layer's filters=
        for j in range(i - 1, 0, -1):
            if "filters=" in lines[j]:
                filters_indices.append(j)
                break

# Print positions for manual verification
print("✅ [yolo] layers found at lines:", yolo_indices)
print("✅ filters= found at lines:", filters_indices)


✅ [yolo] layers found at lines: [606, 692, 779]
✅ filters= found at lines: [602, 688, 775]


In [ ]:
# Path to YOLOv3 config files
cfg_path = "/content/darknet/cfg/yolov3.cfg"
new_cfg_path = "/content/darknet/cfg/yolov3_custom.cfg"

# Read the file
with open(cfg_path, "r") as f:
    lines = f.readlines()

# Define correct positions
filters_indices = [602, 688, 775]  # Filters before `[yolo]`
yolo_indices = [606, 692, 779]  # `[yolo]` layers for `classes=11`

# Apply fixes
for i in range(len(lines)):
    # Fix `filters=48` ONLY in the required places
    if "filters=" in lines[i]:
        if i in filters_indices:
            lines[i] = "filters=48\n"  # Correct filters before `[yolo]`
        else:
            lines[i] = "filters=255\n"  # Reset incorrect filters everywhere else
    # Fix `classes=11` in `[yolo]` layers
    if "classes=" in lines[i]:
        lines[i] = "classes=11\n"  # Ensure correct number of classes

# Save the fixed config file
with open(new_cfg_path, "w") as f:
    f.writelines(lines)

print("✅ Final Fix Applied! Check the output below:")

# Verify the update
!cat /content/darknet/cfg/yolov3_custom.cfg | grep "filters="
!cat /content/darknet/cfg/yolov3_custom.cfg | grep "classes="


✅ Final Fix Applied! Check the output below:
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=48
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=48
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=255
filters=48
classes=11
classes=11
classes=11
